In [14]:
import os
import torch
import transformers
import peft
import datasets
import evaluate
import time
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [15]:
dataset_path = "/app/datasets/oberon/qa/bb_ru/questions-qwen2.5-coder_7b-2025-21-1--09-23-51.json"
model_name = 'bb_ru_Cotype-Nano_20250123-100550'
model_path = f'/app/models/{model_name}'
base_model_name = 'MTSAIR/Cotype-Nano'
EPOCHS = 7
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
#bnb_config= None

In [16]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)

In [17]:
peft_config = peft.PeftConfig.from_pretrained(model_path)
model = transformers.AutoModelForCausalLM.from_pretrained(base_model_name, device_map=device,quantization_config=bnb_config, torchscript=True)
model = peft.PeftModel.from_pretrained(model, model_path, is_trainable=False)

Some weights of Qwen2ForCausalLM were not initialized from the model checkpoint at MTSAIR/Cotype-Nano and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
merged_model = model.merge_and_unload()